# 02 Working with NetCDF Files

We already had a very quick introduction on NetCDF files in the very first chapter of this book. Now we will get to know some more details!

## 02_01 What is a NetCDF File?

Most of today's meteorological data are stored in the NetCDF format (``*.nc``), a format that supports the creation, access, and sharing of array-oriented scientific data. We will, from now on, work with this type of data storage.

In the python basics chapter, we introduced numpy arrays as a new tool for arithmetics and vector computation. Since the climate data we work with is always multi-dimensional and can be stored in such an array, you might think that numpy arrays were the perfect tool to store our data. So why do we still use NetCDF Files for storing climatological and meteorological data? What is the main difference to a normal numpy array?

NetCDF Files can be read by the python module `xarray`! And those xarrays are pretty cool:

Xarray introduces labels in the form of dimensions, coordinates and attributes on top of raw NumPy-like multidimensional arrays. This allows for a more intuitive, more concise, and less error-prone developer experience. 
That means: In a numpy array you could only store e.g. the temperature data of all latitudes and longitudes for a certain time in an array. However, real-world datasets are usually more than just raw numbers: how would you know, where each of the elements in your array is located in a spatial grid? Here comes xarray: you can store a temperature field **and** a label which encodes information about how the array values map to locations in the spatial grid! 

So know all of a sudden, an axis that would have been called 0,1,2,... in a numpy array gets a meaningful name such as "longitude", "latitude" or "time"! That helps us a lot when dealing with this huge amount of data stored in NetCDF files. Still, they behave like normal arrays, which means you can do normal arithmetics and vector computations with them!

Another advantage of an xarray in comparison to numpy arrays is, that you can store metadata on your object, e.g. what unit your data has or who provided the dataset,...
Furthermore, a NumPy array can only have one data type, while NetCDF can hold heterogeneous data in one array.

We will be able to read the NetCDF files in a python interpreter which will transform them into a xarray and do computations and plots with them. 
Let's get started and see how that works!

## 02_02 NetCDF Data - How to get it, read it and understand it

### 02_02_01 Importing the Modules and some Options

Before we can start to look at our data, we will always need to import our three most important modules: `matplotlib`, `numpy` and `xarray`. We will do some more presettings, so everything will look well sized and nice in our notebook.

In [ ]:
# Display the plots in the notebook:
%matplotlib inline

In [ ]:
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary, needed to display world maps
import cartopy.crs as ccrs  # Projections list
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are to ignore certain warnings.
import warnings
warnings.filterwarnings('ignore')

### 02_02_01 Get NetCDF Data

The data we are going to use in this chapter is from ECMWF. We will use a dataset, downloaded with the methods explained in the chapter 02. 

The dataset contains the monthly means of daily means of the 2 meter temperature as well as the total precipitation. You can either get the data [here]() or download it on your own, following the instructions of the previous chapter!

### 02_02_02 Read NetCDF Data

NetCDF files are binary files, which means that you can't just open them in a text editor. You need a special reader for it. 

To handle NetCDF data, python comes with the module [xarray](http://xarray.pydata.org/en/stable/). We already imported it before as `xr`! Xarray provides a lot of useful methods. The one to read NetCDF files is `xr.open_dataset(datadir)`. See the example underneath to understand how it works!

In [ ]:
dataDIR = './data/ERA5-LowRes-MonthlyAvg-t2m_tp.nc' 
# the path to which you saved the netCDF file in the step before
# Here I downloaded the file in the "data" folder which I placed in the same folder as the notebook --> the dot "." 
# in the beginning means "look in the current foler"

ds = xr.open_dataset(dataDIR) # the data of the netCDF File will be stored in "ds" (dataset)

**Note**: you'll have to give an absolute or relative path to the file for this to work. For example ``'C:\PATH\TO\FILE\ERA5-MonthlyAvg-2tm_tp-75-rolled.nc'`` in windows.

Let's see, what our **ds** object looks like!

In [ ]:
ds

Alright. So let's go through this step by step. 

Our `ds`is of the type `xarray.DataSet` (convince yourself above). A`xarray.Dataset` generally consists of the following key properties: 

* **Dimensions**: dimension names for each axis (in our case:'latitude', 'longitude', 'month'), specifies the number of elements for each Dimension, e.g. `time: 488`.
* **Coordinates**: a container of arrays (coordinates) that label each point. E.g.: 1-dim arrays of numbers (like a coordinate vector) , DateTime objects (for time labeling), or strings. On the right hand side you can see the actual values that the coordinates have.
* **Variables**: a numpy.ndarray holding the array’s values, this is where the actual data is stored! In our case, we can expect three arrays of size [241, 480, 488].
* **Attributes**: does not contain any data, is a container that holds arbitrary metadata (attributes), like the title of the data, additional information about the dataset,...

A `xarray.DataSet` is a collection of `xarray.DataArray`s. Each NetCDF file contains such a DataSet.

So what is a `xarray.DataArray`?
It is a multi-dimensional array with labeled or named dimensions. DataArray objects add metadata such as dimension names, coordinates, and attributes (defined below) to underlying “unlabeled” data structures such as our normal numpy arrays.

In our example above, each `xarray.DataArray` would consist of one of the listed Data Variables, e.g. `t2m` the 2 meter temperature. Together those three DataArrays form a `xarray.DataSet`, which in turn is stored in our downloaded netCDF file. 


The **xarray logo** gives us a visual understanding of how a xarray Dataset looks like:

![4D-Data: Data of a specific area dependent on height z and time](xarraylogo.png "")

![](dataset-diagram.png "")

For us, the most interesting two properties will be the coordinates and the variables. Let's have a closer look at them! 

#### Coordinates

You can adress all the different properties of the `xarray.DataArray` via the dot `.` expression:

In [ ]:
ds.time # adress the coordinate 'time'

**Time** goes from 1979 to 2019 and has a resolution of one month. You can read this out of the data listed to the right of `time` at the coordinates. The type of the given values is datetime64.

The **spatial coordinates** are as easy to understand:

In [ ]:
ds.latitude

Latitude goes from 90 to -90 and has the unit 'degrees north'. The spatial resolution of this dataset is 0.75°, as you can easily see from the given values. 

In [ ]:
ds.longitude

#### Variables

As for coordinates, variables can also be accessed directly from the dataset via the `.` syntax! By doing this, you will actually extract one DataArray from the whole DataSet. We can try it now for e.g. the ` t2m` variable.

In [ ]:
ds.t2m

The **attributes** of a variable are extremely important, they cary the *metadata* and must be specified by the data provider. Here we can read in which units the variable is defined (K for Kelvin), as well as a description of the variable (the "long_name" attribute), and sometimes also what the valid value range is (not here). 

From the upper description "... values with dtype=float32", we can also see the data type of the values of our data: in our case this is "float32", a floating point number with 32 bits. 

In [ ]:
ds.tp

The total precipitation is given in m! Since mm is the common unit for precipitation data, we will need to calculate that in order to get expected values of precipitation.

In [ ]:
tp_mm = ds.tp  / 1000

In [ ]:
ds.tp.long_name # adress the attributes of a variable via a second '.'!

## 02_03 NetCDF Data - How to work with it

Let's do some first, simple analyses with our XArray-Dataset!

### Mean of a variable along a certain dimension 

A lot of times, you will want to average your data along a certain dimension axis: Right now the data has three dimensions (latitude, longitude, month). To be able to plot the data (2- or 1-dimensional!), you will need to average over at least one of those three dimensions. This is very easy: there is a method `.mean(dim = 'the_dimension_you_want_to_average_over')` attached to your DataSet variables. So you can just type: `ds.variable_name.mean(dim = '...')`! 

Let's first try to **average over one dimension**, e.g. time. This means, we will get a DataSet of mean sealevel pressure for all the lons and lats, but averaged over the whole period from 1979 to 2019!

In [ ]:
t2m_avg = ds.t2m.mean(dim = 'time')
t2m_avg

As expected, the time coordinate disappears! Instead, we get a new, two-dimensional xarray DataArray of dimensions [latitude, longitude]. We can plot this, to make sure we understood, what the new data looks like:

In [ ]:
t2_tavg = ds.t2m.mean(dim='time')
t2c_tavg = t2_tavg - 273.15
ax = plt.axes(projection=ccrs.Robinson())
t2c_tavg.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='inferno', center=False, 
              vmin=-40, vmax=20, levels=7, cbar_kwargs={'label':'°C'}) 
ax.set_title('Average annual 2m air temperature, ERA5 1979-2019')
ax.coastlines(); ax.gridlines(); 

We are looking at the mean 2m temperature, expressed in ° Celcius. Such time averages are often written with a bar on top of them:

$\overline{msl} = temporal\_mean(msl)$

Don't worry about plotting right now, we will deal with this later. The plots are just here, to give you a visual understanding of our data.


Let's try the same thing for the longitudinal dimension:

In [ ]:
ds.t2m.mean(dim ='longitude') # we are left with two dimensions: the latitudinal coordinate axis and how the TOA
                                    # Shortwave Flux for All-Sky Conditions evolves with time for each latitude

You can also build the **average over two dimensions:**

In [ ]:
ds.t2m.mean(dim = ['time', 'longitude'])

In [ ]:
zonal_t2m_avg = ds.t2m.mean(dim = 'time').mean(dim = 'longitude') # this gives the same output as the command above!

This leaves us with only the meridional component of our dataset, which means we calculated the average over time and all longitudes for each latitude. This is also called the "zonal average".

**"Zonal average"** means "along a latitude circle". It is often writen with ``[]`` or ``<>`` in formulas:

$\left[ \overline{msl} \right] = zonal\_mean(temporal\_mean(msl))$

Note that the two operators are commutative, i.e.:

$\left[ \overline{msl} \right] = \overline{\left[ msl \right]}$


In [ ]:
ds_zavg = ds.t2m.mean(dim=['time', 'longitude'])
t2m_zavg = ds_zavg - 273.15
t2m_zavg.plot();
plt.xlim([-90, 90]);
plt.title('2 meter temperature');
plt.ylabel('°C');

If you do not specify, over which dimension you want to average, you will get the **average over all your dimensions**. The result of this will be a scalar: The mean sealevel pressure over all longitudes, latitudes and over the whole year.

In [ ]:
ds.t2m.mean()

So the msl over all longitudes, latitudes and over the whole year is 278.36!
Well, this means we have an average temperature of ca. 5° Celcius on our planet earth. That is obviously way to cold: the global average air temperature at the Earth surface is about 14°C . 
The error is due to the fact that we averaged over latitudes!

### Averaging over Latitudes

Whenever you **average over latitudes**, you will need to **take into account that our planet is a sphere** and do arithmetics before you calculate the average. **Otherwise** you will get **wrong results**! Go to chapter 08_AveragingoverSphere, to see how this works. 

### Maxima and Minima

You might not only want to calculate an average of your dataset. You can for example also calculate the **minimum** or **maximum** value of your dataset! Let's try it: 

In [ ]:
ds_min = ds.t2m.min()
ds_max = ds.t2m.max()
print(ds_min-273.15)
print(ds_max-273.15)

As for the average, you can also search for a **minimum or maximum value along a certain axis**:

In [ ]:
ds.t2m.min(dim = 'longitude') - 273.15

This gives me the minimum values  of all longitudes for each latitude and each month. 

Let's also have a look at the minimum values of all longitudes and all year, for each latitude:

In [ ]:
ds.t2m.min(dim = ['longitude', 'time']) - 273.15

There are more methods attached to the DataSet variables: e.g. [standard deviation](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.std.html),...

### Data Selection based on the Coordinate Axes

We cannot only average over dimensions, but also select some fix value of a specific axis, e.g. a certain longitude and latitude!

One of the things we'd also like to do is **select** certain years, which is an easy task with xarray and the method `.sel()`:

In [ ]:
ds_avg_lon = ds.t2m.mean(dim ='longitude')
ds_avg_lon.sel(time = '2008') # select the year 2008

In [ ]:
ds_avg_lon_08 = ds.t2m.mean(dim = 'longitude').sel(time = '2008')
# You can also do both, the averaging and the selection in just one step! The result is the same.
# this is possible for all kinds of methods attached to our dataset ds. If you want to plot in the same step, just 
# attach .plot()!
# Anyway, to keep everything ordered, it is sometimes better to do it in single steps than to create one veeeeery long
# command that nobody understands lateron.

Let's see what we have created:

In [ ]:
ds_avg_lon_08 = ds_avg_lon_08 - 273.15;
ds_avg_lon_08.T.plot()
plt.ylim([-90, 90]);


In [ ]:
ds.t2m.sel(time = "2006-12") # we can even select a special month of a year, this makes the time coordinate vanish

In [ ]:
ds.t2m.sel(time = slice('1999','2006')) # or select a series of years, with a slice object!

**Note**: in the code above, we used new python syntax! The **slice** object. A slice object is used for indexing in python and always conists of three parameters: start, stop, step. The start value gives python the index, where it should start selecting the data, stop where it should stop and step, in which steps the data should be indexed. Step is 1 by default. We can also set it to another value, e.g. 2: 

In [ ]:
t2m_selection = ds.t2m.sel(time = slice('1999', '2006', 2)) # select the values of the years 1999-2006 in steps of 2 
                                                            # --> only every second month!
t2m_selection

In [ ]:
t2m_selection.time

As for time, you could also select a certain area of the globe via a slice object:

In [ ]:
region = ds.t2m.mean(dim = 'time').sel(longitude = slice(-20,50)) #select all longitudes from -20° to 50°E

ax = plt.axes(projection=ccrs.PlateCarree()) 
region = region - 273.15
region.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.add_feature(cartopy.feature.BORDERS); 
ax.coastlines();
plt.xlim([-20,50]);

In [ ]:
ds.t2m.latitude


Be careful, to slice the data in the correct way: e.g. slicing the longitude via `slice(40,-20)` will not work, since the data is listed from -180 to 180°E and not from 180 to -180°E. 

However, if we have a closer look at the latitudinal coordinate axis, we see: latitude goes from 90°N to -90°S! Therefore slicing over latitudes works the following: `latitude = slice(40,-40)`!

There is another **problem** that you will encounter **when selecting**: If you try to select values across the 180°E meridian, this will not work! Let's try it:

In [ ]:
ds.t2m.sel(longitude = slice(160.5,-160.5)) # we try to select from 160.5 to -160.5 --> across the 180° meridian!

We get `longitude: 0`! Obviously this did not work. That is, because we tried to select longitude values across the borders of the coordinate axis array. This does not work.

Let's try something else:


In [ ]:
ds.roll(longitude = 1)

If we now look at the longitude coordinate axis, we notice that it changed from starting at -180°E to starting from 179.25°E! We "rolled" the axes, which means we rotated all variables. This is done by the `ds.roll(dim = offset)` syntax. In our case, the dimension is the longitude axis and the offset was 1. 

We can also roll with an offset of e.g. 50:

In [ ]:
ds_rolled = ds.roll(longitude = 50)
ds_rolled

And now try again to select longitudes from 160 to -160°E (from 160.5 to , since 160°E is not an existing coordinate):

In [ ]:
ds_rolled.sel(longitude = slice(160.5,-160.5))

Now it works! We solved the problem of selecting across the borders of the coordinate axis array.

One last example of selection based on the coordinate axes!
Lets plot temperature for the coordinates of Innsbruck( more or less since we have only 0.75° resolution) for the year 2018.

**Note:** If you do not want to type in the correct coordinates (e.g. you don't know if a 160.0° exist or if it is 160.5°), you can instead specify the keyword argument `method` as `nearest`. This will automatically select the values of coordinates, that are nearest to the ones you specified:


In [ ]:
ibk_18 = ds.t2m.sel(time = '2018', latitude = 47, longitude = 11, method = 'nearest')-273.15 
# instead of typing 47.25 and 11.25, just type the integer value and "method = 'nearest'"
ibk_18

In [ ]:
#This is Innsbruck! We selected three dimensions here: lat and lon of ibk and the year 2018
ibk_18.plot()
plt.title('mean t2m Innsbruck 2018(47.25°, 11.25°)');
plt.xlim(['2018-01', '2018-12']);

**Notice**, that selecting data in xarray is way easier than in numpy! When we index with numpy, we need stuff like `a[1:3:2]`,...! Now we can simply tell the array that we want all the values of a certain month or a certain latitude. If we were in a numpy environment, we would have to know, which axis is associated with e.g. the time and then tell numpy somthing similar to: `a[:,1]`. This is one of the things that make xarray so very convenient, without us even noticing it!


### Selection based on a condition

What if we are interested into air temperature on land only, and want to remove the oceans from our analyses? For this we are going to have to "mask out" the oceans grid points. First, we will need to open the "invariant" file: 

In [ ]:
nc_inv = xr.open_dataset('./data/ERA5_75_invariant.nc')
nc_inv

In [ ]:
nc_inv.lsm

In this file, all the ocean gridpoints have the value `0` and all the land gridpoints the value `1`. We will use this information, to "mask out" all the ocean values!


In [ ]:
masked_t2m = ds.t2m.mean(dim = 'time').where(nc_inv.lsm == 1)

If we want to select based on a condition, we take the `.where(data == value)` syntax! Let's see what we have created:

In [ ]:
masked_t2m = masked_t2m - 273.15 # convert into celcius
ax = plt.axes(projection=ccrs.Robinson())
masked_t2m.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.add_feature(cartopy.feature.BORDERS); 
ax.coastlines();

### Resampling Data

Another important tool is resampling data. Resampling is the operation of changing the sampling of the data, i.e. the frequency at which it is sampled. For example you could change the sampling frequency from monthly to yearly. One of the most meaningful way to resample is to do an average, for example over the year:

In [ ]:
res = ds.t2m.resample(time = 'Y').mean()
res

What we get from the resampling operation above is a new dataset, that hast only 36 elements in the time dimension: the 36 years from 1979 to 2014. We averaged the data for each year: `data.resample(time = 'y')` means change the frequency to yearly and `.mean()` says take the mean to get the new values! We can now either change the frequency or the operation by which we resample:

In [ ]:
ds.t2m.resample(time = '5Y').mean() #new frequency: every 5 years

Ways to change the sampling frequency: either add a number in front of the `Y` (as above), or change the `Y` e.g. to `M` for monthly sampling or `D` for daily sampling. 

In [ ]:
ds.t2m.resample(time = 'Y').std() # `.std()` gives us the standard deviation instead of the mean!

Other possible operations for resampling would be: `.max()` bzw. `.min()` if you want all the maximas/minimas or `.sum()` if you want to sum up all the single values. This would make sense for precipitation in mm. 

### GroupBy: Compute monthly/seasonal Climatographies

Another way to look at time series data is to average them according to the time of year to study the annual cycle. This is done with the `.groupby()` method:

In [ ]:
grouped = ds.t2m.groupby('time.month')
grouped

We get a DataArray, grouped into the different months. Now we can apply an average and what we will get is the average t2m of each month over the period 1979-2014!

In [ ]:
grouped.mean()

Note that with groupby, you can also compute seasonal averages:

In [ ]:
ds.t2m.groupby('time.season').mean(dim='time')

We learned a lot about working with datasets and doing analysis. Now, let's finally move on to plotting!